# CSV to Knowledge Graph: DataFrame Transformation with Maplib

## Overview

This notebook demonstrates **DataFrame-native knowledge graph construction** using Maplib and OTTR templates. Unlike traditional RAG approaches that chunk and embed tabular data, we preserve structure by transforming DataFrames directly into RDF knowledge graphs.

## The Challenge: Tabular Data in RAG Systems

When faced with structured data (CSVs, DataFrames, databases), most RAG systems:
- 🔪 **Chunk** the data into text snippets
- 🎲 **Embed** using vector models
- 🔍 **Search** with fuzzy similarity
- 🎯 **Lose** the exact relationships and structure

But tabular data already HAS structure! Why destroy it?

## The Solution: DataFrame → Knowledge Graph

**Maplib + OTTR templates** enable:
- ✅ **Structure preservation** - Keep relationships intact
- ✅ **Type safety** - OTTR validates transformations
- ✅ **Semantic precision** - SPARQL for exact queries
- ✅ **DataFrame-native** - Stay in Polars workflow
- ✅ **Blazing fast** - Rust-backed, zero-copy operations

## The Stack Nobody's Heard Of

- **Maplib**: Rust-backed RDF construction library (from IoT world)
- **OTTR**: W3C template standard for tabular → RDF transformation
- **Polars**: Fast DataFrame library with Apache Arrow backend
- **RDF/SPARQL**: W3C standards for knowledge graphs

## Breaking Best Practices

❌ Not using LangChain  
❌ Not using vector embeddings for tabular data  
❌ Not using Neo4j  
❌ Using tools from IoT/industrial domains  

✅ Preserving structure  
✅ Using formal ontologies  
✅ Querying with precision  

---

Let's transform some jaguars.


## Step 1: Install and Import Dependencies

Install Maplib and Polars if you haven't already:

```bash
pip install maplib polars
```

These are the only dependencies needed for DataFrame → Graph transformation.


In [59]:
import polars as pl
from maplib import Model
from pathlib import Path

print("✅ Imports successful!")
print(f"Polars version: {pl.__version__}")


✅ Imports successful!
Polars version: 1.35.2


## Step 2: Load the CSV Data

Our source data is a simple CSV file with jaguar conservation records:

| jaguar_id | name | gender | location | monitoring_org | first_sighted | is_killed | cause_of_death | identification_mark |
|-----------|------|--------|----------|----------------|---------------|-----------|----------------|---------------------|
| ElJefe | El Jefe | Male | Arizona | AZGFD | 2011-11-19 | false | | Distinctive rosette pattern |

This is **structured data** - it has columns, types, and clear relationships. Perfect for knowledge graphs, terrible for vector embeddings.


In [60]:
# Load CSV as Polars DataFrame
csv_path = "data/jaguars.csv"
df = pl.read_csv(csv_path)

print(f"📊 Loaded {len(df)} jaguar records")
print(f"\nColumns: {', '.join(df.columns)}")

# Create IRI columns (Maplib requires full IRIs for IRI-type parameters)
RES_PREFIX = "http://example.org/resource#"
df = df.with_columns(
    (pl.lit(RES_PREFIX) + pl.col("jaguar_id")).alias("id"),
    (pl.lit(RES_PREFIX) + pl.col("location")).alias("location_iri"),
    (pl.lit(RES_PREFIX) + pl.col("monitoring_org")).alias("monitoring_org_iri")
)

# Select columns matching template parameter order
df_mapped = df.select([
    "id",
    "name", 
    "gender",
    "location_iri",
    "monitoring_org_iri",
    "first_sighted",
    "is_killed",
    "cause_of_death",
    "identification_mark"
])

print(f"\nFirst 3 rows (prepared for mapping):")
df_mapped.head(3)


📊 Loaded 8 jaguar records

Columns: jaguar_id, name, gender, location, monitoring_org, first_sighted, is_killed, cause_of_death, identification_mark

First 3 rows (prepared for mapping):


id,name,gender,location_iri,monitoring_org_iri,first_sighted,is_killed,cause_of_death,identification_mark
str,str,str,str,str,str,bool,str,str
"""http://example.org/resource#El…","""El Jefe""","""Male""","""http://example.org/resource#Ar…","""http://example.org/resource#AZ…","""2011-11-19""",false,null,"""Distinctive rosette pattern re…"
"""http://example.org/resource#Ma…","""Macho B""","""Male""","""http://example.org/resource#Ar…","""http://example.org/resource#AZ…","""1996-01-01""",true,"""Euthanized due to irreversible…","""Distinct left flank mark resem…"
"""http://example.org/resource#So…","""Sombra""","""Male""","""http://example.org/resource#Ar…","""http://example.org/resource#AZ…","""2016-11-01""",false,null,"""Shadow-like spot pattern on sh…"


## Step 3: Initialize Maplib Model and Load Ontology

The Maplib `Model` is your in-memory knowledge graph.

Unlike GraphDB or Neo4j, there's:
- ❌ No server to start
- ❌ No HTTP requests
- ❌ No Docker containers
- ❌ No database configuration

Just pure Python, powered by Rust.

We load the **ontology** first - it defines the classes and properties (the schema).


In [61]:
# Initialize Maplib model
model = Model()

# Load ontology
ontology_path = "data/jaguar_ontology.ttl"
model.read(ontology_path, format="turtle")

print("🚀 Maplib model initialized")
print("   Backend: Rust")
print("   Storage: In-memory")
print("   DataFrame library: Polars")
print(f"\n🧬 Loaded ontology from {ontology_path}")
print("\n✅ Model ready for transformation!")


🚀 Maplib model initialized
   Backend: Rust
   Storage: In-memory
   DataFrame library: Polars

🧬 Loaded ontology from data/jaguar_ontology.ttl

✅ Model ready for transformation!


In [62]:
# Auto-generate template from DataFrame structure
# The "id" parameter tells Maplib which column contains the subject IRI
template_auto = model.map_default(df_mapped, "id")

print("📋 Auto-generated OTTR template:")
print(template_auto)
print("\n✅ DataFrame mapped to RDF automatically!")


📋 Auto-generated OTTR template:
<urn:maplib_default:default_template_0> [
     <http://ns.ottr.xyz/0.4/IRI> ?id, 
     <http://www.w3.org/2001/XMLSchema#string> ?name, 
     <http://www.w3.org/2001/XMLSchema#string> ?gender, 
     <http://ns.ottr.xyz/0.4/IRI> ?location_iri, 
     <http://ns.ottr.xyz/0.4/IRI> ?monitoring_org_iri, 
     <http://www.w3.org/2001/XMLSchema#string> ?first_sighted, 
     <http://www.w3.org/2001/XMLSchema#boolean> ?is_killed, 
    ? <http://www.w3.org/2001/XMLSchema#string> ?cause_of_death, 
     <http://www.w3.org/2001/XMLSchema#string> ?identification_mark ] :: {
  ottr:Triple(?id, <urn:maplib_default:name>, ?name) ,
  ottr:Triple(?id, <urn:maplib_default:gender>, ?gender) ,
  ottr:Triple(?id, <urn:maplib_default:location_iri>, ?location_iri) ,
  ottr:Triple(?id, <urn:maplib_default:monitoring_org_iri>, ?monitoring_org_iri) ,
  ottr:Triple(?id, <urn:maplib_default:first_sighted>, ?first_sighted) ,
  ottr:Triple(?id, <urn:maplib_default:is_killed>, ?is_killed

### What Just Happened?

**Look at the template above!** Maplib:
- Detected IRIs vs literals vs booleans
- Created parameters with proper XSD types
- Generated `ottr:Triple()` statements for each column
- Marked optional parameters (like `cause_of_death`) with `?`

**The catch:** Predicates are generic (`urn:maplib_default:gender` instead of `ont:hasGender`)

Let's query it first to see what we get!


In [63]:
# Query the auto-generated graph
query_generic = """
SELECT ?id ?name ?gender ?location WHERE {
    ?id <urn:maplib_default:name> ?name .
    ?id <urn:maplib_default:gender> ?gender .
    ?id <urn:maplib_default:location_iri> ?location .
}
LIMIT 5
"""

result = model.query(query_generic)
print("🔍 Querying with generic predicates:")
result


🔍 Querying with generic predicates:


id,name,gender,location
str,str,str,str
"""<http://example.org/resource#A…","""Asa""","""Female""","""<http://example.org/resource#P…"
"""<http://example.org/resource#C…","""Cochise""","""Male""","""<http://example.org/resource#A…"
"""<http://example.org/resource#E…","""El Jefe""","""Male""","""<http://example.org/resource#A…"
"""<http://example.org/resource#F…","""Ferinha""","""Female""","""<http://example.org/resource#P…"
"""<http://example.org/resource#M…","""Macho B""","""Male""","""<http://example.org/resource#A…"


## Step 5: Refine Template with Semantic Predicates ✨

**The Workflow: From Auto to Semantic**

```
1️⃣ map_default(df, "id")
   ↓
   Generates template with generic predicates
   (urn:maplib_default:name, urn:maplib_default:gender, etc.)

2️⃣ Copy template structure
   ↓
   Take parameter types and structure

3️⃣ Replace predicates with semantic ones
   ↓
   urn:maplib_default:gender → ont:hasGender
   urn:maplib_default:location_iri → ont:occursIn

4️⃣ Add template to model and map
   ↓
   Same structure, semantic meaning!
```

**Key Insight**: `map_default()` gives you the **scaffolding** (types, structure).  
You provide the **semantics** (meaningful predicates from your ontology).

Let's create a refined template based on the auto-generated structure:


In [64]:
# Create refined template with semantic predicates
# Based on auto-generated structure, but with ontology-aligned property names
template_refined = """
@prefix ont: <http://example.org/ontology#> .
@prefix res: <http://example.org/resource#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ont:JaguarInstanceRefined [
  <http://ns.ottr.xyz/0.4/IRI> ?id,
  <http://www.w3.org/2001/XMLSchema#string> ?name,
  <http://www.w3.org/2001/XMLSchema#string> ?gender,
  <http://ns.ottr.xyz/0.4/IRI> ?location_iri,
  <http://ns.ottr.xyz/0.4/IRI> ?monitoring_org_iri,
  <http://www.w3.org/2001/XMLSchema#string> ?first_sighted,
  <http://www.w3.org/2001/XMLSchema#boolean> ?is_killed,
  ? <http://www.w3.org/2001/XMLSchema#string> ?cause_of_death,
  <http://www.w3.org/2001/XMLSchema#string> ?identification_mark
] :: {
  ottr:Triple(?id, a, ont:Jaguar),
  ottr:Triple(?id, rdfs:label, ?name),
  ottr:Triple(?id, ont:scientificName, "Panthera onca"),
  ottr:Triple(?id, ont:hasGender, ?gender),
  ottr:Triple(?id, ont:occursIn, ?location_iri),
  ottr:Triple(?id, ont:monitoredByOrg, ?monitoring_org_iri),
  ottr:Triple(?id, ont:hasMonitoringStartDate, ?first_sighted),
  ottr:Triple(?id, ont:wasKilled, ?is_killed),
  ottr:Triple(?id, ont:causeOfDeath, ?cause_of_death),
  ottr:Triple(?id, ont:hasIdentificationMark, ?identification_mark),
  ottr:Triple(?id, ont:hasDietType, res:CarnivoreDiet)
} .
"""

print("📋 Refined OTTR template with semantic predicates created!")
print("\n🔄 Key improvements:")
print("   urn:maplib_default:name → rdfs:label")
print("   urn:maplib_default:gender → ont:hasGender")
print("   urn:maplib_default:location_iri → ont:occursIn")
print("   urn:maplib_default:monitoring_org_iri → ont:monitoredByOrg")
print("\n✨ Plus added: rdf:type, scientificName, dietType")


📋 Refined OTTR template with semantic predicates created!

🔄 Key improvements:
   urn:maplib_default:name → rdfs:label
   urn:maplib_default:gender → ont:hasGender
   urn:maplib_default:location_iri → ont:occursIn
   urn:maplib_default:monitoring_org_iri → ont:monitoredByOrg

✨ Plus added: rdf:type, scientificName, dietType


In [65]:
# Create a new model for refined mapping
model_refined = Model()
model_refined.read(ontology_path, format="turtle")

# Add the refined template
model_refined.add_template(template_refined)

# Map the data using the refined template
print(f"🔄 Transforming {len(df_mapped)} rows with semantic template...")
model_refined.map("http://example.org/ontology#JaguarInstanceRefined", df_mapped)

print("✅ Data mapped with refined semantic template!")
print("\n🎉 Your DataFrame is now a semantic knowledge graph!")


🔄 Transforming 8 rows with semantic template...
✅ Data mapped with refined semantic template!

🎉 Your DataFrame is now a semantic knowledge graph!


## Step 7: Query with Semantic SPARQL 🎯

Now we can query with **ontology-aligned predicates** - this is proper semantic querying!

### Query 1: Count All Jaguars


In [66]:
query = """
PREFIX ont: <http://example.org/ontology#>
SELECT (COUNT(?jaguar) as ?count) WHERE {
    ?jaguar a ont:Jaguar .
}
"""

result = model_refined.query(query)
print("📊 Total jaguars in knowledge graph:")
result


📊 Total jaguars in knowledge graph:


count
u32
8


### Query 2: List All Jaguar Names and Genders


In [67]:
query = """
PREFIX ont: <http://example.org/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?gender ?location WHERE {
    ?jaguar a ont:Jaguar .
    ?jaguar rdfs:label ?name .
    OPTIONAL { ?jaguar ont:hasGender ?gender . }
    OPTIONAL { ?jaguar ont:occursIn ?loc . ?loc rdfs:label ?location . }
}
ORDER BY ?name
"""

result = model_refined.query(query)
print("🐆 Jaguar roster:")
result


🐆 Jaguar roster:


name,gender,location
str,str,str
"""Asa""","""Female""",null
"""Cochise""","""Male""",null
"""El Jefe""","""Male""",null
"""Ferinha""","""Female""",null
"""Macho B""","""Male""",null
"""Mick""","""Male""",null
"""Sombra""","""Male""",null
"""Yo'oko""","""Male""",null


### Query 3: Find Killed Jaguars and Causes


In [68]:
query = """
PREFIX ont: <http://example.org/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?cause WHERE {
    ?jaguar a ont:Jaguar .
    ?jaguar ont:wasKilled true .
    OPTIONAL { ?jaguar rdfs:label ?name . }
    OPTIONAL { ?jaguar ont:causeOfDeath ?cause . }
}
"""

result = model_refined.query(query)
print("💔 Jaguars that were killed:")
result


💔 Jaguars that were killed:


name,cause
str,str
"""Macho B""","""Euthanized due to irreversible…"
"""Yo'oko""","""Poached in Sonora Mexico in 20…"


In [69]:
query = """
PREFIX ont: <http://example.org/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?org ?location ?firstSighted WHERE {
    ?jaguar a ont:Jaguar .
    ?jaguar rdfs:label ?name .
    ?jaguar ont:monitoredByOrg ?orgRes .
    ?orgRes rdfs:label ?org .
    ?jaguar ont:occursIn ?locRes .
    ?locRes rdfs:label ?location .
    OPTIONAL { ?jaguar ont:hasMonitoringStartDate ?firstSighted . }
}
ORDER BY ?org ?location
"""

result = model_refined.query(query)
print("🔗 Jaguars with organizational and location context:")
result


🔗 Jaguars with organizational and location context:


name,org,location,firstSighted
str,str,str,str


## Step 8: Export the Knowledge Graph

You can export your knowledge graph to Turtle format for:
- Importing into GraphDB or other triple stores
- Sharing with collaborators
- Version control
- Backup


In [70]:
# Export to Turtle
output_path = "data/exported_jaguars.ttl"
model_refined.write(output_path, format="turtle")

print(f"💾 Knowledge graph exported to {output_path}")
print("\n📤 You can now import this into GraphDB or other triple stores")


💾 Knowledge graph exported to data/exported_jaguars.ttl

📤 You can now import this into GraphDB or other triple stores


---

## Alternative Approach: Load Pre-Written OTTR Template from File

The auto-generate → refine workflow above is perfect for exploration and iteration.

But if you already have a **pre-written OTTR template file** (e.g., `jaguar_template.ottr`), you can skip the generation step and load it directly.

This is useful when:
- You have established templates to reuse
- You're working in production with stable schemas
- You want version-controlled templates

### Load OTTR Template from File


In [71]:
# Create a new model for file-based template approach
model_file = Model()
model_file.read(ontology_path, format="turtle")

# Load OTTR template from file
template_path = "data/jaguar_template.ottr"
with open(template_path, 'r', encoding='utf-8') as f:
    ottr_template = f.read()

# Add template to model
model_file.add_template(ottr_template)

print(f"📋 Loaded OTTR template from {template_path}")
print(f"   Template length: {len(ottr_template)} characters")
print("\n📌 Template defines:")
print("   - ont:JaguarInstance (full data)")
print("   - ont:JaguarBasic (minimal data)")
print("   - ont:JaguarRehab (rescue/release data)")


📋 Loaded OTTR template from data/jaguar_template.ottr
   Template length: 1344 characters

📌 Template defines:
   - ont:JaguarInstance (full data)
   - ont:JaguarBasic (minimal data)
   - ont:JaguarRehab (rescue/release data)


### Map DataFrame with File Template


In [72]:
# Map DataFrame to RDF using template from file
template_iri = "http://example.org/ontology#JaguarInstance"

print(f"🔄 Transforming {len(df_mapped)} rows with file template...")
model_file.map(template_iri, df_mapped)
print("✅ Transformation complete!")

# Quick query to verify
result = model_file.query("""
PREFIX ont: <http://example.org/ontology#>
SELECT (COUNT(?j) as ?count) WHERE { ?j a ont:Jaguar . }
""")
print("\n🎉 File template approach works too!")
result


🔄 Transforming 8 rows with file template...
✅ Transformation complete!

🎉 File template approach works too!


count
u32
8


## Bonus: Load Additional Instance Data

You can also load additional RDF data that wasn't created from CSV.

This shows the power of RDF: **multiple data sources, single graph**.


In [ ]:
# Load existing instance data into the refined model
instances_path = "data/jaguars.ttl"
model_refined.read(instances_path, format="turtle")

print(f"🐆 Loaded additional instances from {instances_path}")
print("\n🔗 Now your graph contains:")
print("   - CSV-derived jaguars (from Maplib transformation)")
print("   - Manually curated jaguars (from jaguars.ttl)")
print("   - All queryable together!")

# Count total jaguars now
result = model_refined.query("""
PREFIX ont: <http://example.org/ontology#>
SELECT (COUNT(?j) as ?total) WHERE { ?j a ont:Jaguar . }
""")
print("\n📊 Total jaguars after merging:")
result


---

## Summary: Auto-Generate vs. Pre-Written Templates

| Aspect | Auto-Generate → Refine | Load from File |
|--------|----------------------|----------------|
| **Setup Time** | Fast (seconds) | Instant (if file exists) |
| **Property Names** | Start generic → refine to semantic | Semantic from start |
| **Type Safety** | Auto-inferred → manual refinement | Fully controlled |
| **Ontology Compliance** | Requires refinement step | Perfect alignment (if well-written) |
| **Use Case** | Exploration, new datasets | Production, established schemas |
| **Flexibility** | See structure, iterate quickly | Need to edit file to change |

## Recommended Workflow:

```
1️⃣ New dataset?
   → Use map_default() to explore
   → Refine template with semantic predicates
   → Save refined template to file

2️⃣ Production pipeline?
   → Load pre-written template from file
   → Map all incoming data consistently
   → Version control your templates
```

## The Power of This Approach:

✅ **Stay in DataFrames** - No need to learn new tools  
✅ **Preserve structure** - Don't destroy relationships with embeddings  
✅ **Query with precision** - SPARQL > fuzzy vector search  
✅ **Blazing fast** - Rust-backed, zero-copy operations  
✅ **Standards-based** - W3C OTTR, RDF, SPARQL  

**This is what structured Graph RAG should look like.** 🎯
